In [102]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

<p>
    Here we are going to get weekly data and we are going to fill missing gdp values.
</p>

In [103]:
covid_clean = pd.read_csv("../Data/Raw/owid-covid-data-clean.csv")
covid_clean.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,life_expectancy,human_development_index,population,excess_mortality,total_cases_smoothed,total_deaths_smoothed,total_boosters_smooth,people_fully_vaccinated_smooth,total_vaccinations_smooth,total_people_vaccinated_smooth
0,AFG,Asia,Afghanistan,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,...,64.83,0.511,41128772.0,-1000.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AFG,Asia,Afghanistan,2020-01-06,0.0,0.0,0.0,0.0,0.0,0.0,...,64.83,0.511,41128772.0,-1000.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AFG,Asia,Afghanistan,2020-01-07,0.0,0.0,0.0,0.0,0.0,0.0,...,64.83,0.511,41128772.0,-1000.0,0.0,0.0,0.0,0.0,0.0,0.0
3,AFG,Asia,Afghanistan,2020-01-08,0.0,0.0,0.0,0.0,0.0,0.0,...,64.83,0.511,41128772.0,-1000.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AFG,Asia,Afghanistan,2020-01-09,0.0,0.0,0.0,0.0,0.0,0.0,...,64.83,0.511,41128772.0,-1000.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
covid_clean['date'] = pd.to_datetime(covid_clean['date'])

In [105]:
reference_date = pd.to_datetime('2020-01-05')
date_diff = (covid_clean['date'] - reference_date).dt.days
covid_clean_new = covid_clean[date_diff % 7 == 0].copy()

In [106]:
covid_clean_new.reset_index(drop=True, inplace=True)
covid_clean_new.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,life_expectancy,human_development_index,population,excess_mortality,total_cases_smoothed,total_deaths_smoothed,total_boosters_smooth,people_fully_vaccinated_smooth,total_vaccinations_smooth,total_people_vaccinated_smooth
0,AFG,Asia,Afghanistan,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,...,64.83,0.511,41128772.0,-1000.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AFG,Asia,Afghanistan,2020-01-12,0.0,0.0,0.0,0.0,0.0,0.0,...,64.83,0.511,41128772.0,-1000.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AFG,Asia,Afghanistan,2020-01-19,0.0,0.0,0.0,0.0,0.0,0.0,...,64.83,0.511,41128772.0,-1000.0,0.0,0.0,0.0,0.0,0.0,0.0
3,AFG,Asia,Afghanistan,2020-01-26,0.0,0.0,0.0,0.0,0.0,0.0,...,64.83,0.511,41128772.0,-1000.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AFG,Asia,Afghanistan,2020-02-02,0.0,0.0,0.0,0.0,0.0,0.0,...,64.83,0.511,41128772.0,-1000.0,0.0,0.0,0.0,0.0,0.0,0.0


In [107]:
covid_clean_new.columns

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'reproduction_rate', 'icu_patients',
       'hosp_patients', 'weekly_icu_admissions', 'weekly_hosp_admissions',
       'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated',
       'total_boosters', 'new_vaccinations_smoothed',
       'new_people_vaccinated_smoothed', 'stringency_index',
       'population_density', 'median_age', 'aged_65_older', 'aged_70_older',
       'gdp_per_capita', 'extreme_poverty', 'cardiovasc_death_rate',
       'diabetes_prevalence', 'female_smokers', 'male_smokers',
       'handwashing_facilities', 'hospital_beds_per_thousand',
       'life_expectancy', 'human_development_index', 'population',
       'excess_mortality', 'total_cases_smoothed', 'total_deaths_smoothed',
       'total_boosters_smooth', 'people_fully_vaccinated_smooth',
       'total_vaccinations_smooth', 'total_pe

In [108]:
for column in covid_clean_new.columns:
    if column == 'excess_mortality':
        per = len(covid_clean_new[covid_clean_new[column] == -1000.0])/len(covid_clean_new)
    else:
        per = len(covid_clean_new[covid_clean_new[column] == -1])/len(covid_clean_new)
    print(f'{column} : {per:.2f}')
# As you can see below, we keep some important featuers 
# and also those that have a lot of lost, will drop from
# data table.

iso_code : 0.00
continent : 0.00
location : 0.00
date : 0.00
total_cases : 0.00
new_cases : 0.00
new_cases_smoothed : 0.00
total_deaths : 0.00
new_deaths : 0.00
new_deaths_smoothed : 0.00
reproduction_rate : 0.18
icu_patients : 0.84
hosp_patients : 0.85
weekly_icu_admissions : 0.91
weekly_hosp_admissions : 0.87
total_vaccinations : 0.08
people_vaccinated : 0.08
people_fully_vaccinated : 0.09
total_boosters : 0.16
new_vaccinations_smoothed : 0.08
new_people_vaccinated_smoothed : 0.08
stringency_index : 0.22
population_density : 0.09
median_age : 0.16
aged_65_older : 0.19
aged_70_older : 0.17
gdp_per_capita : 0.18
extreme_poverty : 0.47
cardiovasc_death_rate : 0.17
diabetes_prevalence : 0.13
female_smokers : 0.38
male_smokers : 0.38
handwashing_facilities : 0.59
hospital_beds_per_thousand : 0.26
life_expectancy : 0.02
human_development_index : 0.20
population : 0.00
excess_mortality : 0.48
total_cases_smoothed : 0.00
total_deaths_smoothed : 0.00
total_boosters_smooth : 0.00
people_fully_

In [109]:
for column in covid_clean_new.columns:
    if column != 'excess_mortality':
        covid_clean_new[column] = covid_clean_new[column].replace(-1, -1000.0)

In [110]:
covid_clean_new = covid_clean_new[['iso_code', 'continent', 'location', 'date',
                                   'new_cases', 'new_deaths', 'new_vaccinations_smoothed',
                                   'new_people_vaccinated_smoothed', 'reproduction_rate',
                                   'stringency_index', 'excess_mortality', 'gdp_per_capita',
                                   'population_density', 'median_age', 'aged_65_older',
                                   'aged_70_older',
                                   'cardiovasc_death_rate',
                                   'diabetes_prevalence', 'female_smokers', 'male_smokers',
                                   'hospital_beds_per_thousand',
                                   'life_expectancy', 'human_development_index', 'population']]
covid_clean_new.head()

,iso_code,continent,location,date,new_cases,new_deaths,new_vaccinations_smoothed,new_people_vaccinated_smoothed,reproduction_rate,stringency_index,...,aged_65_older,aged_70_older,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,hospital_beds_per_thousand,life_expectancy,human_development_index,population
0,AFG,Asia,Afghanistan,2020-01-05,0.0,0.0,0.0,0.0,1.0,0.0,...,2.581,1.337,597.029,9.59,-1000.0,-1000.0,0.5,64.83,0.511,41128772.0
1,AFG,Asia,Afghanistan,2020-01-12,0.0,0.0,0.0,0.0,1.0,0.0,...,2.581,1.337,597.029,9.59,-1000.0,-1000.0,0.5,64.83,0.511,41128772.0
2,AFG,Asia,Afghanistan,2020-01-19,0.0,0.0,0.0,0.0,1.0,0.0,...,2.581,1.337,597.029,9.59,-1000.0,-1000.0,0.5,64.83,0.511,41128772.0
3,AFG,Asia,Afghanistan,2020-01-26,0.0,0.0,0.0,0.0,1.0,0.0,...,2.581,1.337,597.029,9.59,-1000.0,-1000.0,0.5,64.83,0.511,41128772.0
4,AFG,Asia,Afghanistan,2020-02-02,0.0,0.0,0.0,0.0,1.0,0.0,...,2.581,1.337,597.029,9.59,-1000.0,-1000.0,0.5,64.83,0.511,41128772.0


In [111]:
tmp = covid_clean_new[covid_clean_new['gdp_per_capita'] == -1000.0]['iso_code'].tolist()
tmp_series = pd.Series(tmp)
unique_iso_codes = tmp_series.unique()
set(unique_iso_codes)

{'AIA',
 'AND',
 'ASM',
 'BES',
 'BLM',
 'COK',
 'CUB',
 'CUW',
 'FLK',
 'FRO',
 'GGY',
 'GIB',
 'GLP',
 'GRL',
 'GUF',
 'GUM',
 'IMN',
 'JEY',
 'LIE',
 'MAF',
 'MCO',
 'MNP',
 'MSR',
 'MTQ',
 'MYT',
 'NCL',
 'NIU',
 'PCN',
 'PRK',
 'PYF',
 'REU',
 'SHN',
 'SOM',
 'SPM',
 'SYR',
 'TCA',
 'TKL',
 'VAT',
 'VGB',
 'VIR',
 'WLF'}

In [112]:
null_gdp_countries = {
'ASM' : 0,
'AND' : 0,
'AIA' : 0,
'BES' : 0,
'COK' : 0,
'CUB' : 0,
'CUW' : 0,
'FRO' : 0,
'GUF' : 0,
'PYF' : 0,
'GIB' : 0,
'GRL' : 0,
'GLP' : 0,
'GUM' : 0,
'GGY' : 0,
'IMN' : 0,
'JEY' : 0,
'LIE' : 0,
'MTQ' : 0,
'MYT' : 0,
'MCO' : 0,
'MSR' : 0,
'NCL' : 0,
'NIU' : 0,
'MNP' : 0,
'PCN' : 0,
'REU' : 0,
'BLM' : 0,
'SHN' : 0,
'SPM' : 0,
'SOM' : 0,
'SYR' : 0,
'TKL' : 0,
'TCA' : 0,
'WLF' : 0
}

In [113]:
gdp_countries = pd.read_csv("../Data/GDP data for empty countries.csv", encoding='latin1')
gdp_countries.head()

,iso code,Country/Territory,GDP (USD),GDP per Capita (USD)
0,ASM,American Samoa,$871 million1,"$19,673"
1,AND,Andorra,$3.35 billion2,"$41,992"
2,AIA,Anguilla,-,"$28,561"
3,BES,"Bonaire, Sint Eustatius and Saba",-,"$27,000"
4,COK,Cook Islands,-,"$18,806"


In [114]:
# Assuming 'df' is your DataFrame and 'column_name' is the name of the column containing the data
gdp_list = gdp_countries['GDP per Capita (USD)'].str.replace('[€£$]',
                                                             '').str.replace(',',
                                                                             '').astype(float)

C:\Users\MOHAMM~1\AppData\Local\Temp/ipykernel_14804/743333043.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  gdp_list = gdp_countries['GDP per Capita (USD)'].str.replace('[€£$]',


In [115]:
for i, key in enumerate(null_gdp_countries.keys()):
    null_gdp_countries[key] = gdp_list[i]
null_gdp_countries

{'ASM': 19673.0,
 'AND': 41992.0,
 'AIA': 28561.0,
 'BES': 27000.0,
 'COK': 18806.0,
 'CUB': 7449.0,
 'CUW': 20502.0,
 'FRO': 66979.0,
 'GUF': 17928.0,
 'PYF': 22000.0,
 'GIB': 71787.0,
 'GRL': 57116.0,
 'GLP': 24300.0,
 'GUM': 35600.0,
 'GGY': 52761.0,
 'IMN': 79530.0,
 'JEY': 45320.0,
 'LIE': 197504.0,
 'MTQ': 29353.32,
 'MYT': 12204.0,
 'MCO': 239437.0,
 'MSR': 16461.0,
 'NCL': 35745.0,
 'NIU': 19464.0,
 'MNP': 17300.0,
 'PCN': 2429.0,
 'REU': 18517.68,
 'BLM': 42113.52,
 'SHN': 7800.0,
 'SPM': 46200.0,
 'SOM': 592.0,
 'SYR': 421.0,
 'TKL': 7445.0,
 'TCA': 24918.0,
 'WLF': 10908.0}

In [116]:
for key in null_gdp_countries.keys():
    # Filter the rows where 'iso_code' matches the current key
    filtered_rows = covid_clean_new['iso_code'] == key
    
    # Assign values from null_gdp_countries[key] to the filtered rows
    covid_clean_new.loc[filtered_rows, 'gdp_per_capita'] = null_gdp_countries[key]


In [117]:
tmp = covid_clean_new[covid_clean_new['gdp_per_capita'] == -1000.0]['iso_code'].tolist()
tmp_series = pd.Series(tmp)
unique_iso_codes = tmp_series.unique()
unique_iso_codes

array(['VGB', 'FLK', 'PRK', 'MAF', 'VIR', 'VAT'], dtype=object)

In [118]:
final_iso_codes = [34246.0, 70800.0, 639.0, 20321.0, 41976.0, 21198.0]
len(final_iso_codes)

6

In [119]:
for i, key in enumerate(unique_iso_codes):
    # Filter the rows where 'iso_code' matches the current key
    filtered_rows = covid_clean_new['iso_code'] == key
    
    # Assign values from null_gdp_countries[key] to the filtered rows
    covid_clean_new.loc[filtered_rows, 'gdp_per_capita'] = final_iso_codes[i]


In [120]:
tmp = covid_clean_new[covid_clean_new['gdp_per_capita'] == -1000.0]['iso_code'].tolist()
tmp_series = pd.Series(tmp)
unique_iso_codes = tmp_series.unique()
len(unique_iso_codes)

C:\Users\MOHAMM~1\AppData\Local\Temp/ipykernel_14804/4057842459.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tmp_series = pd.Series(tmp)


0

In [121]:
iso_code_gdp_per_capita_df = covid_clean_new.drop_duplicates(subset=['iso_code', 'gdp_per_capita'])[['iso_code', 'gdp_per_capita']]

In [122]:
iso_code_gdp_per_capita_df.to_csv('../Data/Raw/new_approach/iso_code_gdp_per_capita.csv', index=False)

In [123]:
covid_clean_new = covid_clean_new.rename(columns={'new_vaccinations_smoothed': 'new_vaccinations',
                                                  'new_people_vaccinated_smoothed' : 'new_people_vaccinated'})

In [124]:
covid_clean_new = covid_clean_new.drop('location', axis=1)
covid_clean_new = covid_clean_new.drop('continent', axis=1)
covid_clean_new = covid_clean_new.drop('gdp_per_capita', axis=1)

In [125]:
covid_clean_new.head()

,iso_code,date,new_cases,new_deaths,new_vaccinations,new_people_vaccinated,reproduction_rate,stringency_index,excess_mortality,population_density,...,aged_65_older,aged_70_older,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,hospital_beds_per_thousand,life_expectancy,human_development_index,population
0,AFG,2020-01-05,0.0,0.0,0.0,0.0,1.0,0.0,-1000.0,54.422,...,2.581,1.337,597.029,9.59,-1000.0,-1000.0,0.5,64.83,0.511,41128772.0
1,AFG,2020-01-12,0.0,0.0,0.0,0.0,1.0,0.0,-1000.0,54.422,...,2.581,1.337,597.029,9.59,-1000.0,-1000.0,0.5,64.83,0.511,41128772.0
2,AFG,2020-01-19,0.0,0.0,0.0,0.0,1.0,0.0,-1000.0,54.422,...,2.581,1.337,597.029,9.59,-1000.0,-1000.0,0.5,64.83,0.511,41128772.0
3,AFG,2020-01-26,0.0,0.0,0.0,0.0,1.0,0.0,-1000.0,54.422,...,2.581,1.337,597.029,9.59,-1000.0,-1000.0,0.5,64.83,0.511,41128772.0
4,AFG,2020-02-02,0.0,0.0,0.0,0.0,1.0,0.0,-1000.0,54.422,...,2.581,1.337,597.029,9.59,-1000.0,-1000.0,0.5,64.83,0.511,41128772.0


In [126]:
covid_clean_new['date'] = pd.to_datetime(covid_clean_new['date'])
covid_clean_new['week_no'] = (((covid_clean_new['date']-covid_clean_new['date'].min()).dt.days)//7 + 1)

In [127]:
covid_clean_new = covid_clean_new.drop('date', axis=1)

In [128]:
columns = covid_clean_new.columns.tolist()
columns = [columns[-1]] + columns[:-1]
covid_clean_new = covid_clean_new[columns]

In [129]:
covid_clean_new.head()

,week_no,iso_code,new_cases,new_deaths,new_vaccinations,new_people_vaccinated,reproduction_rate,stringency_index,excess_mortality,population_density,...,aged_65_older,aged_70_older,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,hospital_beds_per_thousand,life_expectancy,human_development_index,population
0,1,AFG,0.0,0.0,0.0,0.0,1.0,0.0,-1000.0,54.422,...,2.581,1.337,597.029,9.59,-1000.0,-1000.0,0.5,64.83,0.511,41128772.0
1,2,AFG,0.0,0.0,0.0,0.0,1.0,0.0,-1000.0,54.422,...,2.581,1.337,597.029,9.59,-1000.0,-1000.0,0.5,64.83,0.511,41128772.0
2,3,AFG,0.0,0.0,0.0,0.0,1.0,0.0,-1000.0,54.422,...,2.581,1.337,597.029,9.59,-1000.0,-1000.0,0.5,64.83,0.511,41128772.0
3,4,AFG,0.0,0.0,0.0,0.0,1.0,0.0,-1000.0,54.422,...,2.581,1.337,597.029,9.59,-1000.0,-1000.0,0.5,64.83,0.511,41128772.0
4,5,AFG,0.0,0.0,0.0,0.0,1.0,0.0,-1000.0,54.422,...,2.581,1.337,597.029,9.59,-1000.0,-1000.0,0.5,64.83,0.511,41128772.0


In [130]:
# Here we are going to put one hot encoding to iso_code:
dummy_df = pd.get_dummies(covid_clean_new['iso_code'], prefix='iso_code')
covid_clean_new = pd.concat([covid_clean_new, dummy_df], axis=1)

In [133]:
covid_clean_new.head()

,week_no,iso_code,new_cases,new_deaths,new_vaccinations,new_people_vaccinated,reproduction_rate,stringency_index,excess_mortality,population_density,...,iso_code_VGB,iso_code_VIR,iso_code_VNM,iso_code_VUT,iso_code_WLF,iso_code_WSM,iso_code_YEM,iso_code_ZAF,iso_code_ZMB,iso_code_ZWE
0,1,AFG,0.0,0.0,0.0,0.0,1.0,0.0,-1000.0,54.422,...,0,0,0,0,0,0,0,0,0,0
1,2,AFG,0.0,0.0,0.0,0.0,1.0,0.0,-1000.0,54.422,...,0,0,0,0,0,0,0,0,0,0
2,3,AFG,0.0,0.0,0.0,0.0,1.0,0.0,-1000.0,54.422,...,0,0,0,0,0,0,0,0,0,0
3,4,AFG,0.0,0.0,0.0,0.0,1.0,0.0,-1000.0,54.422,...,0,0,0,0,0,0,0,0,0,0
4,5,AFG,0.0,0.0,0.0,0.0,1.0,0.0,-1000.0,54.422,...,0,0,0,0,0,0,0,0,0,0


In [134]:
covid_clean_new.to_csv('../Data/Raw/new_approach/Weekly_Covid_Data.csv', index=False)